In [ ]:
def cleanup_scene():
    """Removes existing piecelets and rotation empties directly from data."""
    # No need to select/deselect for this method
    objs_to_remove = [o for o in bpy.data.objects if o.name.startswith("Piece.") or o.name.startswith("ROT_")]

    if objs_to_remove:
        print(f"Removing {len(objs_to_remove)} old objects...")
        for obj in objs_to_remove:
            bpy.data.objects.remove(obj, do_unlink=True) # Use direct removal
        print("Finished removing old objects.")
    else:
        print("No old objects found to remove.")

    # Optional: Clear animation data (use with caution if scene has other animations)
    # for action in bpy.data.actions:
    #     bpy.data.actions.remove(action)

    # Ensure the viewport updates if anything was removed
    bpy.context.view_layer.update()

In [5]:
from collections import deque
f = deque(['a','b','c','d'])
f.appendleft(0)
f[0]
[i for i in enumerate(f)]

[(0, 0), (1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]